## NLP

Даны токсичные и нетоксичные комментарии. Надо расклассифицировать эти комменты.

In [1]:
import os
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.optim import lr_scheduler

import pymorphy2
from pymystem3 import Mystem

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
# pip install datasets

In [4]:
# pip install transformers

In [5]:
# import json

# # Укажите путь к вашему JSON-файлу
# file_path = '/home/jupyter/datasphere/project/train.json'

# try:
#     with open(file_path, 'r') as file:
#         json_data = ""
#         for line in file:
#             json_data += line.strip()
        
#         # Поиск всех JSON объектов и их загрузка
#         while json_data:
#             try:
#                 data, idx = json.JSONDecoder().raw_decode(json_data)
#                 print(data)
#                 json_data = json_data[idx:].strip()
#             except json.JSONDecodeError as e:
#                 print("Ошибка при декодировании JSON:", e)
#                 break
# except Exception as e:
#     print("Произошла ошибка:", e)


In [6]:
# from datasets import load_dataset

# dataset = load_dataset("Hrukanina/ultra-small")

In [7]:
# pip install --upgrade fsspec

In [8]:
# nltk.download('punkt')

In [9]:
# nltk.download('wordnet')

In [10]:
# nltk.download('stopwords')

In [11]:
# nltk.download('averaged_perceptron_tagger_ru')

In [12]:
# pip install torchtext

In [13]:
# pip install pymorphy3

In [14]:
# pip install pymorphy2

In [15]:
# pip install pymystem3

In [16]:
# pip install transformers

In [17]:
# загружаем наш датасет
data_train = pd.read_csv('train.csv.zip', engine='python', sep = ';')
data_train.head()

,id,comment,toxic
0,0,Бесполезно что-то пытаться донести так до люде...,0
1,1,А свиньи разве умеют читать?\n,1
2,2,Червепидорские страны парашной конфедерации -К...,1
3,3,"Нет, это не так. Я зашёл сюда специально за эт...",1
4,4,"Дополнение: Дентрен, та чилийская грязь, на ко...",0


Сделаем дополнительную аугментацию данных для улучшения результатов модели.

In [18]:
# загружаем датасет с https://github.com/s-nlp/russe_detox_2022/blob/main/data/input/train.tsv
d1 = pd.read_csv('train.tsv', engine='python', sep='\t')
d1.head()

,index,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
0,0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст...",Где этот герой был до этого со своими доказате...,"и,где этот герой был до этого со своими доказа..."
1,1,"О, а есть деанон этого петуха?","О, а есть деанон",NaN,NaN
2,2,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка.","Ерунду всякую пишут,из-за этого лайка.",NaN
3,3,из за таких пидоров мы и страдаем,из за таких плохих людей мы и страдаем,Из-за таких людей мы и страдаем,из за таких как он мы и страдаем
4,4,гондон путинский он а не артист,"Человек Путина он, а не артист",NaN,NaN


In [19]:
# https://huggingface.co/datasets/AlexSham/Toxic_Russian_Comments

file_path = 'train.jsonl'
# Чтение файла JSON и преобразование в DataFrame
df = pd.read_json(file_path, lines=True)
df

,text,label
0,"видимо в разных регионах называют по разному ,...",0
1,"понятно что это нарушение правил, писать капсл...",1
2,"какие классные, жизненные стихи....",0
3,а и правда-когда его запретили?...,0
4,в соленой воде вирусы живут .ученые изучали со...,0
...,...,...
223456,вова - дима когда же вы подавитесь деньгами???...,0
223457,"какая красота, просто нет слов выразить чувств...",0
223458,вы пост гаи выставити на перекрестке возле 21 ...,0
223459,как -то на лебедей непохожи,0


In [20]:
# # токсичные комментарии различной тематики https://huggingface.co/datasets/NiGuLa/Russian_Sensitive_Topics
# d3 = pd.read_csv('sensitive_topics.csv', engine='python')
# d3.head()

In [21]:
# преобразуем все датасеты чтобы можно было их объединить

# d3['toxic']=1
# d3['comment'] = d3['text']

df_toxic = pd.DataFrame({'comment': d1['toxic_comment'],
                         'toxic': [1]*len(d1) })

df_non_toxic = pd.DataFrame({'comment': d1['neutral_comment1'] ,
                             'toxic': [0]*len(d1)})

df.columns = ['comment','toxic']

In [22]:
df_combined = pd.concat([df_toxic, df_non_toxic, data_train[['comment','toxic']], df])
# d3[['comment','toxic']],

In [23]:
df_combined

,comment,toxic
0,"и,чё,блядь где этот херой был до этого со свои...",1
1,"О, а есть деанон этого петуха?",1
2,"херну всякую пишут,из-за этого лайка.долбоебизм.",1
3,из за таких пидоров мы и страдаем,1
4,гондон путинский он а не артист,1
...,...,...
223456,вова - дима когда же вы подавитесь деньгами???...,0
223457,"какая красота, просто нет слов выразить чувств...",0
223458,вы пост гаи выставити на перекрестке возле 21 ...,0
223459,как -то на лебедей непохожи,0


In [24]:
df_combined.groupby('toxic').describe()

comment                                                                
        count  unique                                                top freq
toxic                                                                        
0      201302  201294                                            Замолчи    2
1       51055   48803  и,чё,блядь где этот херой был до этого со свои...    2

In [25]:
text = np.array(df_combined.comment.values).tolist()
target = df_combined.toxic.astype(int).values

In [26]:
text[:3]

['и,чё,блядь где этот херой был до этого со своими доказательствами?',
 'О, а есть деанон этого петуха?',
 'херну всякую пишут,из-за этого лайка.долбоебизм.']

# Токенизация

## Собственный токенайзер

In [27]:
def cleanText(string):
    """This function deletes all symbols except Cyrilic and Base Latin alphabet,
    stopwords, functional parts of speech. Returns string of words stem."""
    # Common cleaning
    string = string.lower()
    string = re.sub(r"http\S+", "", string)
    string = re.sub(r'(?:(?!\u0301)[\W\d_])+', ' ', string) # нужно избавиться от нерусских символов
    string = re.sub('[^а-яА-Я]+', ' ', string)
    string = str.replace(string,'Ё','е')
    string = str.replace(string,'ё','е')

    # prog = re.compile('[А-Яа-яA-Za-z]+')
    # words = prog.findall(string.lower())
    tokens = word_tokenize(string)

    
    # Word Cleaning
    ## Stop Words
    stopwords = nltk.corpus.stopwords.words('russian')
    words = [w for w in tokens if w not in stopwords]
    ## Cleaning functional POS (Parts of Speech)
    functionalPos = {'CONJ', 'PRCL'}
    words = [w for w, pos in nltk.pos_tag(words, lang='rus') if pos not in functionalPos]
    
    # Stemming
    stemmer = SnowballStemmer('russian')
    stemmed_words = [stemmer.stem(w) for w in words]

    # lemmatizer = pymorphy3.MorphAnalyzer()
#     lemmatizer = pymorphy2.MorphAnalyzer()

#     stemmed_words = [lemmatizer.parse(w)[0].normal_form for w in words]

#     ЛЕММАТАЙЗЕРЫ работают оооочень долго
    # mystem = Mystem()
    # stemmed_words = [mystem.lemmatize(w) for w in words]
    
    # words = [word_tokenize(comment) for comment in words]
    # stemmed_words = [word_tokenize(comment) for comment in stemmed_words]

    # return ' '.join(stemmed_words)
    return stemmed_words



In [28]:
text_my_tokenization = list(map(cleanText, text))

In [29]:
text_my_tokenization[:3]

[['ч', 'бляд', 'хер', 'сво', 'доказательств'],
 ['деанон', 'петух'],
 ['херн', 'всяк', 'пишут', 'лайк', 'долбоебизм']]

## Предобученный токенайзер Rusvectores

In [30]:
# pip install wget

In [31]:
import zipfile
import wget
model_url = 'http://vectors.nlpl.eu/repository/20/220.zip'
m = wget.download(model_url)
model_file = '220.zip'
# берем модель и распаковываем ее

In [32]:
with zipfile.ZipFile(model_file, 'r') as archive:
    archive.extractall()

In [33]:
import gensim
model_rusvec = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [34]:
def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken

In [35]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False  # переменная для запоминания того, что нам встретилось имя собственное
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10: # если список короткий — строчка не содержит разбора, пропускаем
            continue
        (word_id,token,lemma,pos,xpos,feats,head,deprel,deps,misc) = t
        if not lemma or not token: # если слово пустое — пропускаем
            continue
            # Разметка имен собственных!!!!!!!!!!!!!!!!!!!!
        if pos in entities: # здесь отдельно обрабатываем имена собственные — они требуют особого обращения
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            # обработка для чисел
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct: # обрабатываем случай, когда не надо сохранять пунктуацию (по умолчанию она сохраняется)
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [36]:
# pip install ufal.udpipe

In [37]:
from ufal.udpipe import Model, Pipeline # это пайплайн для обработки модели русвекторс
import os
import re
import sys

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    # print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    # print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
    return '\n'.join(tagged)

In [38]:
text_pretrained_tokenization = [[', '.join(tag_ud(comment).split())] for comment in text[:3]]

In [39]:
text_pretrained_tokenization

[['и_CCONJ, ,чё,блядь_ADV, где_ADV, этот_DET, херый_NOUN, быть_AUX, до_ADP, это_PRON, с_ADP, свой_DET, доказательство_NOUN'],
 ['о_INTJ, а_CCONJ, быть_VERB, деанон_PART, этот_DET, петух_NOUN'],
 ['херн_NOUN, всякий_DET, писать_VERB, ,из-з_ADP, этот_PRON, лайка.долбоебизм_NOUN']]

## Разделение набора

In [40]:
# Разделение на тренировочный (96%) и вспомогательный наборы (10%)
X_train, X_noo, y_train, y_noo = train_test_split(text_my_tokenization, target, test_size=0.04, random_state=42, stratify=target)

# Разделение вспомогательного набора на валидационный (60%) и тестовый наборы (40%)
X_valid, X_test, y_valid, y_test = train_test_split(X_noo, y_noo, test_size=0.4, random_state=42, stratify=y_noo)

## Word2vec через словарь и ручное присвоение индексов

In [41]:
# from collections import Counter
# vocab = Counter()

# for comment in text_my_tokenization:
#     vocab.update(comment)

In [42]:
# len(vocab)

In [43]:
# filtered_vocab = set()
# # удаляем слова из словаря которые встречаются реже чем 21 раз
# for word in vocab:
#     if vocab[word] > 20:
#         filtered_vocab.add(word)

In [44]:
# len(filtered_vocab)

In [45]:
# word2id = {'PAD':0}
# # переименовываем слова в айдишники
# for word in filtered_vocab:
#     word2id[word] = len(word2id)

In [46]:
# # обратное преобразование из айди в слово
# id2word = {i:word for word, i in word2id.items()}

In [47]:
# sentences = [] # комментарий в наших данных
# documents = []

# # преобразуем статьи
# for comment in text_my_tokenizationt:
#     tokens = comment
#     if not tokens:
#         continue
#     documents.append(tokens)
#     ids = [word2id[token] for token in tokens if token in word2id]
#     sentences.append(ids)

In [48]:
# #cbow
# X = []
# y = []

# window = 10
# for sent in sentences:
#     for i in range(len(sent)-1):
#         word = sent[i]
#         context = sent[max(0, i-window):i] + sent[i+1:i+window]

#         X.append(context)
#         y.append(word)

In [49]:
# import tensorflow as tf
# # иксы могут быть разного размера поэтому надо обрезать, либо дополнить недостающую часть
# X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=20, padding='post')
# y_train = np.array(y_train)
# X_valid = tf.keras.preprocessing.sequence.pad_sequences(X_valid, maxlen=20, padding='post')
# y_valid = np.array(y_valid)

In [50]:
# pip install tensorflow

In [51]:
# model_cbow = tf.keras.Sequential()

# model_cbow.add(tf.keras.layers.Embedding(input_dim=len(word2id),
#                                     input_length=20,
#                                     output_dim=30))

# model_cbow.add(tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))) # суммируем по 2й размерности

# model_cbow.add(tf.keras.layers.Dense(len(word2id), activation='softmax'))
# model_cbow.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy'
#               )

In [52]:
# model_cbow.fit(X_train, y_train,
#           validation_data=(X_valid, y_valid),
#           batch_size=1024,
#          epochs=5)

## Word2vec в Gensim + собственная нейросеть

In [68]:
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import gensim

word2vec_model = gensim.models.Word2Vec(text_my_tokenization, vector_size=50, window=10, workers=4, sg = 0)
word2vec_model.train(text_my_tokenization, total_examples=len(text), epochs=10)

class CommentsDataset(Dataset):
    def __init__(self, comments, labels, word2vec_model, max_words=25, max_vec_length=50):
        self.comments = comments
        self.labels = labels
        self.word2vec_model = word2vec_model
        self.input_size = word2vec_model.vector_size
        self.max_words = max_words
        self.max_vec_length = max_vec_length

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = self.comments[idx]
        label = self.labels[idx]

        comment_vec = [self.word2vec_model.wv[word] if word in self.word2vec_model.wv else np.zeros(self.input_size, dtype=np.float32) for word in comment[:self.max_words]]

        # Padding для длины вектора слов
        padded_comment_vec = []
        for vec in comment_vec:
            if len(vec) < self.max_vec_length:
                padded_vec = np.pad(vec, (0, self.max_vec_length - len(vec)), mode='constant')
            else:
                padded_vec = vec[:self.max_vec_length]
            padded_comment_vec.append(padded_vec)

        while len(padded_comment_vec) < self.max_words:
            padded_comment_vec.append(np.zeros(self.max_vec_length, dtype=np.float32))

        return torch.tensor(padded_comment_vec), torch.tensor([label])


# Создание DataLoader для тренировочных данных и вадидационных данных
train_dataset = CommentsDataset(X_train, y_train, word2vec_model)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = CommentsDataset(X_valid, y_valid, word2vec_model)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

test_dataset = CommentsDataset(X_test, y_test, word2vec_model)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [69]:
# Нейронная сеть для классификации

class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=50, hidden_size=64, num_layers=1, batch_first=True)
        # self.lstm2 = nn.LSTM(64, 32, batch_first=True)
        self.fc1 = nn.Linear(64, 32)
        self.fc = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm1(x)
        # out, _ = self.lstm2(out)        
        # out = self.fc(out[:, -1, :])  # Используем только последний выход
        out = self.fc1(out[:, -1, :])
        out = self.fc(out)
        out = self.sigmoid(out)
        return out
    

# Инициализация модели, функции потерь и оптимизатора
model = LSTMModel()
# criterion = nn.BCELoss()
criterion = nn.MSELoss()
# criterion = nn.BCEWithLogitsLoss() 
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# ШЕДУЛЕР
scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

In [70]:
# Обучение модели
num_epochs = 10
valid_loss_min = np.Inf
correct_train = 0
total_train = 0
correct_valid = 0
total_valid = 0

for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    for comment, label in train_loader:
        
        # comments = comment.to(device)
        comment = comment.to(torch.float32)
        # label = label.long()
        label = label.float()
        
        optimizer.zero_grad()
        
        outputs = model(comment)
        # outputs = (outputs > 0.5).float()
        loss = criterion(outputs, label)
        
        loss.backward()
        optimizer.step()
        
        predicted = (outputs > 0.49).float()  # Применяем порог для бинарной классификации
        correct_train += (predicted == label).sum().item()
        total_train += label.size(0)
        train_loss += loss.item() * comment.size(0)
    
    model.eval()
    for comment, label in valid_loader:

        # comment = comment.to(device)
        comment = comment.to(torch.float32)
        # label = torch.unsqueeze(label, 1).float()
        # label = label.long()
        label = label.float()
        
        outputs = model(comment)
        # outputs = (outputs > 0.5).float()
        
        predicted = (outputs > 0.49).float()  # Применяем порог для бинарной классификации
        correct_valid += (predicted == label).sum().item()
        total_valid += label.size(0)
        loss = criterion(outputs, label)
        valid_loss += loss.item() * comment.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    valid_loss = valid_loss / len(valid_loader.dataset)
    train_accuracy = correct_train / total_train
    valid_accuracy = correct_valid / total_valid

    
    print(f'Эпоха {epoch+1}, Тренировочная ошибка: {train_loss}, Валидационная ошибка: {valid_loss}')
    print(f'Эпоха {epoch+1}, Тренировочная точность: {train_accuracy}, Валидационная точность: {valid_accuracy}')

    # СОХРАНЕНИЕ ЛУЧШЕЙ МОДЕЛИ
    if valid_loss < valid_loss_min:
        print('Валидационная ошибка уменьшилась')
        torch.save(model.state_dict(), 'lstm_model.pth')
        valid_loss_min = valid_loss

    scheduler.step()

Эпоха 1, Тренировочная ошибка: 0.04880428247198599, Валидационная ошибка: 0.04229793279553692
Эпоха 1, Тренировочная точность: 0.9388430707250828, Валидационная точность: 0.9438666006273733
Валидационная ошибка уменьшилась
Эпоха 2, Тренировочная ошибка: 0.042405700143792475, Валидационная ошибка: 0.04085199668756241
Эпоха 2, Тренировочная точность: 0.9423950103606838, Валидационная точность: 0.9459303285454845
Валидационная ошибка уменьшилась
Эпоха 3, Тренировочная ошибка: 0.04147371333991551, Валидационная ошибка: 0.0423812728921966
Эпоха 3, Тренировочная точность: 0.9440770735814944, Валидационная точность: 0.9457377139397942
Эпоха 4, Тренировочная ошибка: 0.04165405676266087, Валидационная ошибка: 0.0402006562510147
Эпоха 4, Тренировочная точность: 0.944923264895031, Валидационная точность: 0.9463017995707446
Валидационная ошибка уменьшилась
Эпоха 5, Тренировочная ошибка: 0.03855858711500841, Валидационная ошибка: 0.038714110226100856
Эпоха 5, Тренировочная точность: 0.9462614854991

## Классификация трансфер лернингом, используя Bert

In [56]:
from transformers import BertTokenizer, BertForSequenceClassification

# load tokenizer and model weights
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [57]:
encoded_texts = [tokenizer.encode(comment, return_tensors='pt') for comment in text]

Token indices sequence length is longer than the specified maximum sequence length for this model (696 > 512). Running this sequence through the model will result in indexing errors


In [58]:
target_size = 25
padded_tensors = []

for tensor in encoded_texts:
    if tensor.size(1) > target_size:  # Обрезаем слишком большие тензоры
        new_tensor = tensor[:, :target_size]
    else:  # Дополняем нулями слишком маленькие тензоры
        new_tensor = torch.cat((tensor, torch.zeros(1, target_size - tensor.size(1))), dim=1)
    
    padded_tensors.append(new_tensor)

# Теперь все тензоры в списке comment_tensors имеют одинаковый размер 25

In [67]:
from torch.utils.data import Dataset

class CommentsDataset11(Dataset):
    def __init__(self, comments, labels):
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        return self.comments[idx], self.labels[idx]

    from torch.utils.data import DataLoader


# Создаем датасет
train_dataset = CommentsDataset11(X_train, y_train)
# Загрузка данных с помощью даталоадера
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = CommentsDataset11(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

#### Модель работает очень долго на аугментированных данных

# Предсказание тестовых комментариев

In [60]:
# выгружаем лучшую модель
model = LSTMModel()
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

LSTMModel(
  (lstm1): LSTM(50, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [61]:
data_test = pd.read_csv('test.csv.zip', engine='python', sep = ';')
data_test.head()
test = np.array(data_test.comment.values)
id_test = data_test.id
test = list(map(cleanText, test))

In [64]:
list_of_zeros = [0] * len(test)
test_dataset = CommentsDataset(test, list_of_zeros, word2vec_model)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [65]:
predicted_classes=[]
for comment, _ in test_loader:

    comment = comment.to(torch.float32)
        
    outputs = model(comment)
        
    predicted = (outputs > 0.49).float()  # Применяем порог для бинарной классификации

    predicted = predicted.cpu().detach().numpy().squeeze()
    predicted_classes.append(predicted)


In [66]:
import itertools
predicted_classes
flat_list = list(itertools.chain(*predicted_classes))
sub = pd.DataFrame({"ID": id_test, "prediction": list(map(int, flat_list))})
sub.to_csv('sample_submission.csv',index=False)
# файл скачается в ту же папку, где находится программный код